In [96]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Composition

In OOP different classes can be related to each other in different ways. One of the most common ways is composition. Composition is a way to combine different classes into a new class. The new class is called a composite class. The composite class is made up of the other classes. The composite class can be used just like any other class. The composite class can have its own attributes and methods. The composite class can also have attributes and methods from the other classes. The composite class can also have attributes and methods that are a combination of the other classes.

## Overlapping generation model

### 1 Consumer: Log utility

Suppose that utility for individuals born at time $ t $ takes the form


<a id='equation-eq-crra'></a>
$$
U_t = u(c_t) + \beta u(c_{t+1}) \tag{23.1}
$$

Here

- $ u: \log(c)$  
- $ \beta \in (0, 1) $ is the discount factor  
- $ c_t $ is time $ t $ consumption of the individual born at time $ t $  
- $ c_{t+1} $ is time $ t+1 $ consumption of the same individual  

Solving for saving, we get


<a id='equation-saving-log-2-olg'></a>
$$
s_t = s(w_t, R_{t+1}) = \frac{\beta}{1+\beta} w_t \tag{23.6}
$$

Saving turns into investment. Assume 100% capital depreciation so that $s_t$ turns into $k_{t+1}$. In optimum, 

$$
k_{t+1} = s(w_t, R_{t+1}) = \frac{\beta}{1+\beta} w_t \tag{23.6b}
$$

Also consumers are capital owner. Producer actually rents capital from consumers. So, consumers receive rental income from producers.

In [97]:
class Consumer_LogUtility:
    def __init__(self, beta):
        self.beta = beta
    def foc_c(self, knext, w, R): # (23.6b)
        return knext - (self.beta/(1+self.beta))*w


### 2 Producer: Cobb-Douglas

For each integer $ t \geq 0 $, output $ y_t $ in period $ t $ is given by
the **Cobb-Douglas production function**


<a id='equation-cobb-douglas'></a>
$$
y_t = k_t^{\alpha} \ell_t^{1-\alpha} \tag{23.7}
$$

Here $ k_t $ is capital, $ \ell_t $ is labor, and  $ \alpha $ is a parameter
(sometimes called the “output elasticity of capital”).

The profit maximization problem of the firm is


<a id='equation-opt-profit-olg'></a>
$$
\max_{k_t, \ell_t} \{ k^{\alpha}_t \ell_t^{1-\alpha} - R_t k_t - \ell_t w_t   \} \tag{23.8}
$$

The first-order conditions are obtained by taking the derivative of the
objective function with respect to capital and labor respectively and setting
them to zero:

$$
(1-\alpha)(k_t / \ell_t)^{\alpha} = w_t
    \quad \text{and} \quad
    \alpha (k_t / \ell_t)^{\alpha - 1} = R_t \tag{23.9}
$$


In [98]:
class Producer_cobbDouglas:
    def __init__(self, alpha):
        self.alpha = alpha
    def foc_l(self, k, l, w): # (23.9)
        return (1-self.alpha)*(k/l)**self.alpha - w
    def foc_k(self, k, l, R): # (23.9)
        return self.alpha*(l/k)**(1-self.alpha) - R


### 3 Dynamic equilibrium

In Macroeconomics, dynamic equilibrium is a set of solutions for **ENDOGENOUS** variables each date that 

- satisfies all FOCs
- satisfies all constraints, 

and is normally presented as a function of parameters and

- predetermined variables, e.g. today's capital stock (predetermined yesterday).

> Dynamical endogenous variables are variables with a time subscript which are not shocks -- there is no shock in the model. In OLG case here, they are $(k_{t+1}, w_t, R_t)$ (other than $c_t$).

Both agents, consumer and producer, have their FOCs equal to zero satisfied. Assume $\ell_t=1$ for all $t$. And $k_0$ is given. Then, we have

$$
\begin{aligned}
    w_t &= (1-\alpha) k_t^{\alpha} \\
    R_t &= \alpha k_t^{\alpha - 1}
\end{aligned}
$$

The dynamics goes as follows:

$$
\begin{aligned}
    k_{t+1} &= s(w_t, R_{t+1}) \\
    &= \frac{\beta}{1+\beta} w_t \\
    &= \frac{\beta}{1+\beta} (1-\alpha) k_t^{\alpha}
\end{aligned}
$$



In [99]:
class OLGMarket:
    def __init__(self, consumer, producer, k0):
        self.consumer = consumer
        self.producer = producer
        self.k0 = k0
        self.k = k0
        self.equilibrium_dynamics = [[k0, None, None]]
    def equilibrium(self):
        """
        return a tuple of (knext, w, R)
        and track the dynamics in self.equilibrium_dynamics
        """ # docstring: documentation string
        
        alpha = self.producer.alpha
        beta = self.consumer.beta

        w = (1-alpha)*self.k**alpha
        R = alpha*(self.k)**(alpha-1)
        knext = (beta/(1+beta))*w

        # keep track of the dynamics
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][1] = w
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][2] = R
        self.equilibrium_dynamics.append([knext, None, None])

        # set the market time to the next period
        self.k = knext
        return (knext, w, R)



In [100]:
consumer = Consumer_LogUtility(0.9)
producer = Producer_cobbDouglas(0.3)
market = OLGMarket(consumer, producer, 1)

In [101]:
print(market.equilibrium.__doc__)


        return a tuple of (knext, w, R)
        and track the dynamics in self.equilibrium_dynamics
        


In [102]:
# initial state
market.k0, market.equilibrium_dynamics

(1, [[1, None, None]])

In [103]:
# dynamic equilibrium
market.equilibrium()

market.k0, market.equilibrium_dynamics

(0.33157894736842103, 0.7, 0.3)

(1, [[1, 0.7, 0.3], [0.33157894736842103, None, None]])

In [ ]:
# simulate the dynamics
for t in range(30):
    market.equilibrium()

market.equilibrium_dynamics

In [89]:
# turn market.equilibrium_dynamics into a DataFrame
import pandas as pd
import copy
df = pd.DataFrame(market.equilibrium_dynamics, columns=['k', 'w', 'R'])
df.head()

,k,w,R
0,1.000000,0.700000,0.300000
1,0.331579,0.502660,0.649696
2,0.238102,0.455121,0.819195
3,0.215584,0.441756,0.878193
4,0.209253,0.437824,0.896708


In [113]:
df.index
df.columns
[x for x in df.columns]

RangeIndex(start=0, stop=31, step=1)

Index(['k', 'w', 'R'], dtype='object')

['k', 'w', 'R']

- `DataFrame.index` is the index of the `DataFrame`. It is a `pd.Index` object.

In [ ]:
# use plotly to plot the dynamics for the first 10 periods
import plotly.express as px
periods = 10
fig = px.line(df[:periods], x=df.index[:periods], y=df.columns)
fig.show()

![](img/dynamic_eq.png)